In [1]:
import obspy
import torch
import os
from glob import glob

## Download event data

In [2]:
!wget -q https://github.com/AI4EPS/PhaseNet/releases/download/test_data/test_data.zip -O test_data.zip
!unzip -q -o test_data.zip


## Run PhaseNet-Plus

In [3]:
mseed_list = glob('test_data/mseed/*.mseed')
with open("mseed_list.txt", "w") as f:
    f.write("\n".join(mseed_list))

In [4]:
ngpu = torch.cuda.device_count()
base_cmd = "../predict.py --model phasenet_plus --data_list mseed_list.txt --result_path ./results --format=mseed  --batch_size 1 --workers 1"

plot_figure = True
if plot_figure:
    base_cmd += " --plot_figure"

In [5]:
if ngpu == 0:
    cmd = f"python {base_cmd} --device cpu"
elif ngpu == 1:
    cmd = f"python {base_cmd}"
else:
    cmd = f"torchrun --nproc_per_node {ngpu} {base_cmd}"

print(cmd)
os.system(cmd);

python ../predict.py --model phasenet_plus --data_list mseed_list.txt --result_path ./results --format=mseed  --batch_size 1 --workers 1 --plot_figure --device cpu
Not using distributed mode
Namespace(model='phasenet_plus', resume='', backbone='unet', phases=['P', 'S'], device='cpu', workers=1, batch_size=1, use_deterministic_algorithms=False, amp=False, world_size=1, dist_url='env://', data_path='./', data_list='mseed_list.txt', hdf5_file=None, prefix='', format='mseed', dataset='das', result_path='./results', plot_figure=True, min_prob=0.3, add_polarity=False, add_event=False, highpass_filter=0.0, response_xml=None, folder_depth=0, cut_patch=False, nt=20480, nx=5120, resample_time=False, resample_space=False, system=None, location=None, skip_existing=False, distributed=False)
Total samples:  ./.mseed : 16 files


Predicting:   0%|          | 0/16 [00:00<?, ?it/s]

## Plot results

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.flatten()

for i, f in enumerate(glob('results/figures_phasenet_plus/*.png')):
    img = Image.open(f) 
    axes[i].imshow(img)
    axes[i].axis('off')
    if i >= 15:
        break
plt.tight_layout()
plt.show()